EXPORT_METAR is script to export metar value from certain station automatically using python. The basic idea is webscrapping content data from web aviation.bmkg.go.id. The output is csv file format.

There are two main part of this script, can change and can't change part. Please don't change the 'can't change part (mark by #)'

created by ulil hidayat (ulil.hidayat@bmkg.go.id)


In [1]:
#import module
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

#CAN CHANGE PART (change station, st_date (start date), time_st (time start), ed_date (end date), time_ed (time end))
station = 'waqt' #icao index station
st_date = ['01', '06', '2023'] # day, month, year  | ex  : 01 june 2023
time_st = ['00', '00'] #hour, minute    | ex : hour 00 minute 00
ed_date = ['30', '06', '2023'] #same w/st_date (1 month)
time_ed = ['01', '00'] #same with time_st
url = f'https://aviation.bmkg.go.id/web/metar_speci.php?icao={station}&sa=yes&fd={st_date[0]}%2F{st_date[1]}%2F{st_date[2]}&fh={time_st[0]}&fm={time_st[1]}&ud={ed_date[0]}%2F{ed_date[1]}%2F{ed_date[2]}&uh={time_ed[0]}&um={time_ed[1]}&f=raw_format'
#CAN CHANGE PART (UNTIL THIS)

# CANT'CHANGE PART (FROM HERE)
def page_count(url, page=None):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.implicitly_wait(3)
    checkbox = driver.find_element(By.CSS_SELECTOR, "input[name='agreement']")
    checkbox.click()
    submit_button = driver.find_element(By.CSS_SELECTOR, "input[name='submit']")
    submit_button.click()

    # Find the dropdown element
    driver.implicitly_wait(3)
    dropdown = Select(driver.find_element(By.XPATH, './/*[@id="main_page"]/p[1]/label/select'))

    # Get the total number of pages
    total_pages = len(dropdown.options)
    #print(total_pages)

    if page is not None:
        page.append(total_pages)
    
def importmetar(url, index, page=None, metar_value=None):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.implicitly_wait(3)
    checkbox = driver.find_element(By.CSS_SELECTOR, "input[name='agreement']")
    checkbox.click()
    submit_button = driver.find_element(By.CSS_SELECTOR, "input[name='submit']")
    submit_button.click()

    # Find the dropdown element
    driver.implicitly_wait(3)
    dropdown = Select(driver.find_element(By.XPATH, './/*[@id="main_page"]/p[1]/label/select'))

    dropdown_xpath = './/*[@id="main_page"]/p[1]/label/select'
    dropdown.select_by_index(index)
    
    #calculate span count
    x = 1
    elements_found = True
    span_count = 0

    while elements_found:
        xpath = f'//*[@id="main_page"]/span[{x}]'
        elements = driver.find_elements(By.XPATH, xpath)
    
        if len(elements) > 0:
            span_count += 1
            x += 1
        else:
            elements_found = False

    #print(f"Total span elements found: {span_count}")

    #Export metar value
    value = []
    for i in range(1,span_count):
        element = driver.find_element(By.XPATH,  f'.//*[@id="main_page"]/span[{i}]')
        value1 = element.text
        # Split the string at the newline character
        split_data = value1.split('\n') #for only take metar data, not with the header 

        # Get the value after the newline character
        value2 = split_data[1]
        value.append(value2)

    #print(value)

    if metar_value is not None:
        metar_value.append(value)

    driver.quit()
#CAN'T CHANGE PART (UNTIL THIS)

#CAN'T CHANGE PART
pages = []
page_count(url, page=pages)

metar_value = []
for i in range(pages[0]):
    importmetar(url,i, metar_value=metar_value)
    
f_metar_val = [item for sublist in metar_value for item in sublist]

#reverse the list
series = pd.Series(f_metar_val)
reversed_series = series[::-1]
f_metar_val_rev = reversed_series.tolist()

df = pd.DataFrame(f_metar_val_rev, columns=['METAR'])
df.to_csv('export_metar.csv', sep=',', index=False)
print ('Script is finish, check the folder for the output file')
#CAN'T CHANGE PART


/tmp/ipykernel_12889/3534003241.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/tmp/ipykernel_12889/3534003241.py:41: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Script is finish, check the folder for the output file
